# Combat stats
This is to determine the average statistics for combat within the game. On average monsters should deal about 1/3 of the player's health in damage. They should instakill a player no more than 2% of the time.

Monsters average stats are calculated using those advertised under Making Monsters. Players stats are calculated by a combination of what average stats would be, and expected equipment purchases (especially for armor).

## Set Averages
The average stats for monsters and players by level.

In [1]:
def dice(rolls, sides):
    return {
        'rolls': rolls,
        'sides': sides
    }

In [2]:
DAMAGE_DICE = {
    1: [dice(1, 8), dice(1, 6)],
    2: [dice(2, 8)],
    3: [dice(1, 10), dice(1, 8)],
    4: [dice(2, 12), dice(1, 4)],
    5: [dice(2, 12), dice(1, 8)],
    6: [dice(1, 20), dice(2, 8)],
    7: [dice(2, 20)],
    8: [dice(2, 20), dice(1, 8)],
    9: [dice(2, 20), dice(1, 12)],
    10:[dice(3, 20)]
    
}

player_damage_dice = [dice(1, 10), dice(1, 4)]

MONSTER_ATTACK_BONUS = {
    1: 4,
    2: 4,
    3: 5,
    4: 7,
    5: 7,
    6: 9,
    7: 12,
    8: 12,
    9: 13,
    10: 14
}

In [3]:
MONSTER_ARMOR = {
    1: 8,
    2: 9,
    3: 11,
    4: 13,
    5: 16,
    6: 18,
    7: 21,
    8: 23,
    9: 26,
    10: 28
}

MONSTER_HEALTH = {
    1: 21,
    2: 26,
    3: 30,
    4: 34,
    5: 38,
    6: 42,
    7: 45,
    8: 48,
    9: 50,
    10: 55
}

In [4]:
def check_pos_int(values):
    """Check if a value is a positive integer
    
    Args:
        values: (List) values to be checked for validity
    """
    for value in values:
        assert value >= 0, 'Value must be a nonnegative integer'

check_pos_int([1, 2, 3])

In [5]:
def player_health(level):
    check_pos_int([level])
    level_ups = level - 1
    health = 15
    health += level_ups * 3
    health += 5 * int(level_ups / 3)
    return health

assert(player_health(1) == 15)
assert(player_health(2) == 18)
assert(player_health(4) == 29)
assert(player_health(9) == 49)
assert(player_health(10) == 57)

In [6]:
def player_armor(level):
    check_pos_int([level])
    if level <= 3:
        return 9
    if level <= 5:
        return 14
    if level <= 6:
        return 15
    if level <= 7:
        return 16
    return 19
    
assert(player_armor(1) == 9)
assert(player_armor(3) == 9)
assert(player_armor(6) == 15)
assert(player_armor(9) == 19)

## Attack
Functions needed to calculate the power of a monster attack.

In [7]:
import random

def get_attack(damage_dice, bonus):
    """ calculate the power of an attack
    
    Args:
        damage_dice (list of damage_die): The dice to be rolled
        bonus (int): The static bonus added to attacks     
    
    Return: (int) power of an attack
    """
    check_pos_int([bonus])
    total = 0
    largest_maxed = 0
    for damage_die in damage_dice:
        sides = damage_die['sides']
        for i in range(damage_die['rolls']):
            roll = random.randint(1, sides)
            total += roll
            if roll == sides:
                largest_maxed = max(largest_maxed, sides)
    if largest_maxed > 0:
       total += random.randint(1, largest_maxed) 
    return total + bonus

for i in range (1000):
    assert(get_attack([dice(1, 4)], 0) in range(1,9))
    assert(get_attack([dice(2, 6)], 0) in range(2,25))
    assert(get_attack([dice(1, 8)], 0) in range(1,17))
    assert(get_attack([dice(2, 12)], 0) in range(2,49))

In [8]:
def damage_percentage(attack, level):
    """Calculate the percentage of health lost
    
    Args:
        All are positive ints
        
    Returns:
        Health remaining after the attack
    """
    check_pos_int([attack, level])
    armor = player_armor(level)
    health = player_health(level)
    if attack <= armor:
        return 0
    damage = attack - armor
    if damage >= health:
        return 100
    return (damage / health) * 100

assert(damage_percentage(12, 1) == 20)
assert(damage_percentage(18, 1) == 60)
assert(damage_percentage(38, 5) == 75)
assert(damage_percentage(42, 8) == 50)

In [11]:
def combat_stats(level):
    '''
    Calculate the stats of combat.
    
    Args:
        Level (int)
    
    Return: (list of float) [average damage dealt, hits that one shotted player, misses]
    '''
    total_damage_percent = 0
    instakills = 0
    missed = 0
    iterations = 100000
    for i in range(iterations):
        attack = get_attack(DAMAGE_DICE[level],
                       MONSTER_ATTACK_BONUS[level])
        damage_percent = damage_percentage(attack, level)
        total_damage_percent += damage_percent
        if damage_percent == 100:
            instakills += 1
        elif damage_percent == 0:
            missed += 1
    stats = [total_damage_percent, instakills, missed]
    return [stat / iterations for stat in stats]

In [12]:
for level in range(1, 11):
    print(combat_stats(level))

[28.298733333332184, 0.01481, 0.20894]
[28.89744444443725, 0.00758, 0.1558]
[33.91104761905039, 0.01433, 0.07371]
[34.877965517245435, 0.00731, 0.05378]
[37.75621875, 0.00699, 0.03018]
[42.7719142857135, 0.0237, 0.01582]
[41.677372093019876, 0.02423, 0.0142]
[43.3654130434806, 0.01954, 0.01069]
[46.80416326530786, 0.02321, 0.00412]
[48.857771929822505, 0.02176, 0.00138]


## Player Attack
Determine the average power of a player's attack based on their level. Strangely enough, only a select few power cards actually increase the damage of a base attack, so it will be assumed that powers do not increase the base damage of the attack.
Assumptions will be made about player stat progression, when players will be crowned in their martial skill, and when they will buy mastercrafted weapons. No other assumptions will be made.

Currently, monster armor will be between the attack of 10% of player attacks and 25% of player attacks. Martial players will on average miss 20% and finesse 25% of their attacks. Specialized martial classes will basically never miss with their buffs active. Low armor buffs blade dancers.

In [31]:
# Constants for player attack

PLAYER_ATTACK_BONUS = {
    1: 6,
    2: 7,
    3: 8,
    4: 8,
    5: 12,
    6: 12,
    7: 16,
    8: 16,
    9: 17,
    10: 17
}

MARTIAL_ATTACK_DICE = [dice(1, 10), dice(1, 4)]
FINESSE_ATTACK_DICE = [dice(1, 8), dice(1, 4)]

In [34]:
def roll_stats(dice):
    """
    Give the power of the average attack. THe bottom 10%, 25%, 75%, 90%, and 95% of attacks.
    
    Args: dice (list of dice)
    
    return (dict)
    """
    iterations = 10000
    total = 0
    attack_values = []
    for i in range(iterations):
        attack = get_attack(dice, 0)
        total += attack
        attack_values.append(attack)
    attack_values.sort()
    stats = {
        'avg': int(total / iterations),
        'fifteen': attack_values[1000],
        'twenty_five': attack_values[2500],
        'seventy_five': attack_values[7500],
        'ninety': attack_values[9000],
        'ninety_five': attack_values[9500]
    }
    return stats

def average_attack_power(dice):
    roll = roll_stats(dice)
    for level in range(1, 11):
        bonus = PLAYER_ATTACK_BONUS[level]
        average = roll['avg'] + bonus
        fifteen = roll['fifteen'] + bonus
        twenty_five = roll['twenty_five'] + bonus
        seventy_five = roll['seventy_five'] + bonus
        ninety = roll['ninety'] + bonus
        ninety_five = roll['ninety_five'] + bonus
        print(f'Level {level} players have attack average: {average} fifteen: {fifteen} twenty-five: {twenty_five} seventy-five:{seventy_five} ninety: {ninety} ninety_five: {ninety_five}')
        
average_attack_power(MARTIAL_ATTACK_DICE)

Level 1 players have attack average: 15 fifteen: 10 twenty-five: 12 seventy-five:17 ninety: 21 ninety_five: 24
Level 2 players have attack average: 16 fifteen: 11 twenty-five: 13 seventy-five:18 ninety: 22 ninety_five: 25
Level 3 players have attack average: 17 fifteen: 12 twenty-five: 14 seventy-five:19 ninety: 23 ninety_five: 26
Level 4 players have attack average: 17 fifteen: 12 twenty-five: 14 seventy-five:19 ninety: 23 ninety_five: 26
Level 5 players have attack average: 21 fifteen: 16 twenty-five: 18 seventy-five:23 ninety: 27 ninety_five: 30
Level 6 players have attack average: 21 fifteen: 16 twenty-five: 18 seventy-five:23 ninety: 27 ninety_five: 30
Level 7 players have attack average: 25 fifteen: 20 twenty-five: 22 seventy-five:27 ninety: 31 ninety_five: 34
Level 8 players have attack average: 25 fifteen: 20 twenty-five: 22 seventy-five:27 ninety: 31 ninety_five: 34
Level 9 players have attack average: 26 fifteen: 21 twenty-five: 23 seventy-five:28 ninety: 32 ninety_five: 35
L

In [ ]:
def dict_by_level(items):
    """
    Create a dictionary organized by level given a list of 10 elements.
    """
    1: items[0],
    2: items[1],
    3: items[2],
    4: items[3],
    5: items[4],
    6: items[5],
    7: items[6],
    8: items[7],
    9: items[8],
    10: items[9]

MONSTER_ARMOR = dict_by_level([11, 12, 13, 14, 17, 18, 21, 22, 22, 24])

# Health per player. Expect it to be about 1 / 3 of the average attack of a player.
MONSTER_HEALTH = dict_by_level([11, 12, 13, 14, 17, 18, 21, 22, 22, 24])

## Player Damage
This is an attempt to calculate how much damage I expect the average player to do on the average turn. This extremely difficult to calculate because of the sheer number of variables. There is no one, for instance, to exactly measure the damage value of positioning and terrain. It is similarly difficult to calculate the value of utitly abilities, such as those of most ancestries.
We will isntead intend to examine the damage benefit of select identity and specialty cards that enchance player abilities in melee combat. These will be used as a baseline to balance the other cards.

In [35]:
# These constants represent several assumptions being made

# Factor to multiply an ability's damage by if it only happens once per period
REST = 1/9
BREAK = 1/6
COMBAT = 1/3

PRIMARY_STATS=[3, 4, 5, 5, 6, 6, 7, 7, 8, 8]
SECONDARY_STATS=[3, 4, 4, 5, 5, 5, 6, 7, 7, 7]

# Reactions and bonus actions are considered free for now
# Double damage is considered a +4 bonus. Double damage with advantage +6

In [38]:
# Create a dict given a list of bonuses and attack multipliers for a power card to have
def card(bonuses, attacks, advantage=False):
    return {
        1: {
            bonus: bonuses[0],
            attack: 1 + attacks[0],
            advantage: advantage
        },
        2: {
            bonus: bonuses[1],
            attack: 1 + attacks[1],
            advantage: advantage
        }
        3: {
            bonus: bonuses[2],
            attack: 1 + attacks[2],
            advantage: advantage
        }
    }

In [ ]:
# Bonuses to certain identity cards based on stars
# Cards have 
BLADE_DANCER = card([3, 4, 4], [0, 0, BREAK * 3])
DUELIST = card([0, 0, 4], [0, 2, 2], True)
ASSASSIN = card([0, 0, 6], [0, 2 * COMBAT, 2 * COMBAT], True)
# Figure out the advantage in damage of MUSCLEMAN attacks vs 1d10 + 1d4 and use that for bonus
MUSCLEMAN = card([]

In [ ]:
# def player_attack(damage_dice, level, advantage=False):
#     total = 0
#     iterations = 10000
#     for i in range(iterations):
#         total += get_attack(damage_dice, (level * 2.5) + 6)
#     return total / iterations


# def player_damage(damage_dice, level, advantage=False):
#     total = 0
#     iterations = 10000
#     monster_armor = monster_armor_by_level[level]
#     for i in range(iterations):
#         attack = get_attack(damage_dice, (level * 2.5) + 6)
#         if advantage:
#             attack = max(attack, get_attack(damage_dice, bonus))
#         total += max(0, attack - monster_armor)
#     return total / iterations

# def rounds_to_victory(level):
#     damage_dice = player_damage_dice
#     iterations = 10000
#     total_rounds = 0
#     attack_bonus = 6 + (level * 2.5)
#     monster_armor = monster_armor_by_level[level]
#     monster_health = monster_health_by_level[level] * 4
#     for i in range(iterations):
#         current_monster_health = monster_health
#         rounds = 0
#         while current_monster_health > 0:
#             attack = get_attack(damage_dice, attack_bonus)
#             damage = max(0, attack - monster_armor)
#             current_monster_health -= damage
#             rounds += .25
#         total_rounds += rounds
#     return total_rounds / iterations

In [ ]:
# # for level in range(1, 10):
#     rounds = rounds_to_victory(level)
#     damage = 6 + (level * 2.5)
#     attack = player_attack(player_damage_dice, level)
#     print(f'Players on average have attack: {attack} deal {damage} and win in {rounds}')

In [ ]:
# def average_roll(dice):
#     iterations = 10000
#     total = 0
#     for i in range(iterations):
#         total += get_attack(dice, 0)
#     return total / iterations

In [ ]:
# print(average_roll([dice(1, 12), dice(1, 8)]))
# print(average_roll([dice(1, 12)]))
# print(average_roll([dice(1, 10), dice(1, 4)]))
# print(average_roll([dice(1, 8), dice(1, 4)]))
# print(average_roll([dice(1, 10)]))
# print(average_roll([dice(1, 10)]))
# print(average_roll([dice(2, 10)]))